# MULTI-AGENT SYSTEM

This system has 2-3 agents:
- coder, this is the one that has to generate a cypher query
- executor, this is the one that test the query on a neo4j database
- retriever, this is the agent that will pass the documents to be used by coder. It cannot be considered as a part of the system because it only initiate the chat with the documents.

In [6]:
from autogen import AssistantAgent, GroupChatManager, GroupChat, config_list_from_json
from autogen.agentchat.contrib.retrieve_user_proxy_agent import RetrieveUserProxyAgent, TEXT_FORMATS
from pathlib import Path
import os

from CypherExecutor import CypherCodeExecutor
from neo4j_tools import save_schema

In [5]:
# Save database schema and then store the path to the directory
save_schema()
p = Path(__name__).parent.resolve() / "schemas"


#### LLM MODEL CONFIGURATION #####

config_list = config_list_from_json(env_or_file="CONFIG_LIST", filter_dict={"model": "gpt-4o"})
config_list[0]["api_key"] = os.environ.get("GITHUB_TOKEN")

llm_config = {"config_list": config_list, "temperature": 0.2}

prompt = """
You are an AI helpful assistant.

### RULES ###
    - You have to explain everything you do.
    - When the question refers to something that is contained 
      in the database you must generate 1 query to retrieve data.
    - When generating a query you must use the Cypher format.

### TERMINATION ###
You can add 'TERMINATE' at the end of your message if:
    - exitcode is 0 and you explained the output.
    - exitcode is 1 and the error is not fixable with another query.
    - there is not a query to execute.

### DATABASE SCHEMA ### 
{input_context}

### QUESTION ###
{input_question}
"""

# This are the accepted formats that can be retrieved by RetrieveUserProxy
print(f'Accepted formats for "docs_path": \n{TEXT_FORMATS}')

In [6]:
def termination_msg(x):
    return isinstance(x, dict) and "TERMINATE" == str(x.get("content", ""))[-9:].upper()


#TODO: useful pdf files at https://neo4j.com/docs/docs-archive/
doc_retriever = RetrieveUserProxyAgent(
    name="doc_retriever",
    is_termination_msg=termination_msg,
    max_consecutive_auto_reply=3,
    human_input_mode="NEVER",
    retrieve_config={
        "docs_path": str(p), # A list of urls, dirs or files can be passed
        "extra-docs": True,
        "model": config_list[0]["model"],
        "get_or_create": True,
        "customized_prompt": prompt
    },
    code_execution_config=False,
    description="Assistant who has extra content retrieval power for solving cypher queries."
)

# Coder generates cypher queries with the help of the db schema 
coder      = AssistantAgent(
    name="coder",
    is_termination_msg=termination_msg,
    system_message=prompt,
    llm_config=llm_config,
)

# Executor runs the query in a jupyter notebook
executor   = AssistantAgent(
    name="executor",
    is_termination_msg=termination_msg,
    human_input_mode="NEVER", # THIS IS NOT SECURE! 
    description= """Executor provides feedback based on the result of the query.""",
    code_execution_config={"executor": CypherCodeExecutor()}
    )

# A question to be translated in cypher query language
PROBLEM  = "Explain with rhymes what are the differences between Pocahontas and Sabrina"

def _reset_agents():
    """
    This function reset all the agents used for the group chat.
    This should be used every time you start a new conversation.
    """
    doc_retriever.reset()
    coder.reset()
    executor.reset()

def state_transition(last_speaker, groupchat):
    """
    This function simply return the next speaker based on the last one.
    In this case we decided that doc_retriever should speak only to initiate chat
    and then the chat is from coder to executor in a sort of loop.
    """
    if   last_speaker is coder: return executor
    elif last_speaker is executor: return coder
    elif last_speaker is doc_retriever: return coder

def rag_chat():
    """
    Resets the agents, creates the group chat and then initiate chat.
    The first agent to speak is the doc_retriever that will only pass:
        - prompt
        - files (like the schema), that is the context
        - question
    """
    _reset_agents()
    groupchat = GroupChat(agents=[doc_retriever, coder, executor], messages=[], 
                          max_round=5, speaker_selection_method=state_transition)
    manager   = GroupChatManager(groupchat=groupchat, llm_config=llm_config)

    doc_retriever.initiate_chat(manager,message=doc_retriever.message_generator, problem=PROBLEM,n_results=3)


In [ ]:
#### START OF THE GROUP CHAT BETWEEN CODER AND EXECUTOR ####
rag_chat()